<a href="https://colab.research.google.com/github/Praise-CS/Text-Summerizer-AI/blob/main/TextSummerizerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is installing the needed libaries


!pip install transformers torch datasets huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Initializes the model name then loads the model and the tokenizer
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Ensure the model is moved to the GPU to accelerate training.
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

cuda


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("giuliadc/cnndm_5k")

# Print available dataset splits
print(dataset)

# Check if there's a 'train' split. If not, create a split from the 'test' dataset
if 'train' in dataset:
    train_dataset = dataset['train']
    eval_dataset = dataset['test']
else:
    # Split the test dataset into train and eval (e.g., 90% train, 10% eval)
    test_dataset = dataset['test']
    dataset_split = test_dataset.train_test_split(test_size=0.1)  # Adjust test_size as needed
    train_dataset = dataset_split['train']
    eval_dataset = dataset_split['test']

print("Columns: ", dataset.column_names)
print("Train dataset size: ", len(train_dataset))
print("Eval dataset size: ", len(eval_dataset))


README.md:   0%|          | 0.00/896 [00:00<?, ?B/s]

cnndm_5k_2-0.630-test.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'text', 'reference-summary', 'source'],
        num_rows: 5000
    })
})
Columns:  {'test': ['id', 'text', 'reference-summary', 'source']}
Train dataset size:  4500
Eval dataset size:  500


In [ ]:
def preprocess_function(examples):
    # Extract the text and reference-summary from the dataset
    inputs = [doc for doc in examples['text']]
    summary = [doc for doc in examples['reference-summary']]

    # Tokenize the input articles with a maximum length of 512 tokens, adding padding if necessary, and truncating longer sequences.
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    # Tokenize the target summaries (summary) with the target tokenizer context, with a max length of 128, adding padding and truncating.
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(summary, max_length=128, padding="max_length", truncation=True, return_tensors="pt")

    # Attach the tokenized target summaries as the labels in the input dictionary for model training.
    model_inputs["labels"] = labels["input_ids"]

    # Transfer the tokenized inputs and labels to the appropriate device (e.g., GPU or CPU).
    model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

    return model_inputs



In [ ]:

# Now apply the tokenization
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Set up the training arguments for the model
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Path to save model checkpoints and outputs
    evaluation_strategy="epoch",         # Perform evaluation at the end of each epoch
    learning_rate=2e-5,                  # Initial learning rate for the optimizer
    per_device_train_batch_size=8,       # Training batch size per device (GPU/CPU)
    per_device_eval_batch_size=8,        # Evaluation batch size per device (GPU/CPU)
    weight_decay=0.01,                   # Weight decay factor to reduce overfitting
    save_total_limit=3,                  # Maximum number of checkpoints to keep
    num_train_epochs=10,                  # Total number of training epochs
    predict_with_generate=True,          # Enable generation of predictions during evaluation
    logging_dir="./logs",                # Directory for storing training logs
    logging_steps=100,                   # Log training progress every 100 steps
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

# Initialize the Seq2SeqTrainer with the specified model and datasets
trainer = Seq2SeqTrainer(
    model=model,                            # The model instance that will be trained
    args=training_args,                     # Training configurations set previously
    train_dataset=tokenized_train_dataset,  # Preprocessed training dataset ready for training
    eval_dataset=tokenized_eval_dataset,    # Preprocessed evaluation dataset for model validation
    tokenizer=tokenizer                     # Tokenizer for processing inputs and generating outputs
)


In [ ]:
##Starts the training
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.385300,1.254231
2,1.361900,1.256616
3,1.351000,1.252439
4,1.336600,1.250693
5,1.340400,1.251327
6,1.323900,1.249692
7,1.352800,1.249928
8,1.323400,1.249552
9,1.324900,1.249336
10,1.345200,1.249272


TrainOutput(global_step=5630, training_loss=1.3460456414502213, metrics={'train_runtime': 2541.6802, 'train_samples_per_second': 17.705, 'train_steps_per_second': 2.215, 'total_flos': 8440916037599232.0, 'train_loss': 1.3460456414502213, 'epoch': 10.0})

In [ ]:
# Evaluate the model on the evaluation dataset and store the metrics
metrics = trainer.evaluate()

# Output the evaluation metrics to assess model performance
print(metrics)


{'eval_loss': 1.2492719888687134, 'eval_runtime': 8.2408, 'eval_samples_per_second': 60.674, 'eval_steps_per_second': 7.645, 'epoch': 10.0}


In [ ]:
#Saving the model
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/spiece.model',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [ ]:
def generate_summary(input_text):
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, padding=True, truncation=True).to(device)
    summary_ids = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage:
input_text = "There were scenes of jubilation in the Dutch city of Dordrecht today, as flag-waving locals turned out to welcome King Willem-Alexander and his glamorous wife Maxima today. The visit, which kicked off with a pageant aboard a flotilla of boats on the local canal, was part of Willem-Alexander's 48th birthday celebrations. Koningsdag - or King's Day - is a public holiday in the Netherlands and is traditionally marked with a royal procession either in the capital Amsterdam or in one of the country's major cities. Happy family: The Dutch royals celebrated King Willem-Alexander's birthday in Dordrecht . Warm welcome: The Dutch royal couple were all smiles as they meet well-wishers in the trading city . Today, it was the turn of Dordrecht, a city in the south-west of the country, which was once a major trading port, as well as a military hub. Signs of its martial past were thin on the ground today, however, with the streets filled with a sea of joyful humanity, most of whom cheered and waved as the royals floated past. Taking centre stage was Argentina-born Queen Maxima, who opted for a typically eye-catching ensemble of a raspberry pink coat and hat enlivened with a pair of chunky crystal earrings. Also aboard the royal barge, the Dordtevaar, were the couple's three daughters: Catharina-Amalia, Princess of Orange, 11, Princess Alexia, nine, and Princess Ariane, eight. Willem-Alexander's brother Prince Constantijn, 45, was also aboard, as was his 48-year-old wife, Princess Laurentien, who opted for a tailored dress in patriotic bright orange. Is that it? Eldest daughter Princess Catharina-Amalia, 11, cuddles up to her mother during the event . Round of applause: The Queen and the princesses join in with a round of applause for Willem-Alexander . Big arrival: The couple and their three daughters travelled through the city by barge . Family affair: The couple was joined by Prince Constantijn and his wife Laurentien who wore bright orange . Looking good: Maxima looked wonderful in her summery raspberry pink get-up . Matching: The Dutch queen completed her look with a slick of bright pink lipstick and chunky earrings . All smiles: The 43-year-old appeared to be enjoying the occasion and smiled brightly throughout . Echoing their parents and uncle, all three girls smiled and waved at the crowds, most of whom seemed to be enjoying the bright spring sunshine while watching the procession go past. Others used the royal day off to make some extra money, with one enterprising soul setting up a stand that offered selfies with cardboard versions of the Dutch royals for €1 apiece. Koningsdag is a long-standing Dutch tradition that began in 1890 with Queen Wilhelmina who celebrated her birthday on the 31st August. When her daughter Juliana became Queen, the date was moved to the 30th April - a tradition maintained by her daughter Beatrix, even though her real birthday is the 31st January. Since April 2013, when Willem-Alexander took the throne following his mother's abdication, the holiday has taken place his birthday, the 27th April. The celebrations, which kick off the night before with King's Parties, also include vrijmarkts - flea markets - in most towns and mass consumption of a delicious orange pastry called the tompouce. A right royal selfie! A well-wisher snaps a selfie with the Dutch King and Queen . Royal wave: King Willem-Alexander waves at the crowds during a walk through the city with his children . Enjoying the sunshine: Both royals and revellers made the most of the beautiful sunshine . Big hug: Queen Maxima hugs her two youngest daughters, Princesses Ariane and Alexia . Speech! King Willem-Alexander talks to the crowds during his visit to Dordrecht . Enterprising: This little girl decided to add to her pocket money by selling selfies for €1 apiece . On the water: The canal pageant was swelled by orange-clad well-wishers on their own boats . Turn out: Willem-Alexander's birthday is a public holiday which meant everyone had the day off . Throng: Huge crowds turned out to see the royal barge drift past . Patriotic: Dutch flags were flying all over the town while many revellers came dressed in patriotic orange ."
summary = generate_summary(input_text)
print("Generated Summary:", summary)

Generated Summary: Dutch royals celebrated King Willem-Alexander's birthday in Dordrecht today. The visit kicked off with a pageant aboard a flotilla of boats on the local canal. The royal barge, the Dordtevaar, was the couple's three daughters.


In [ ]:
# Upload the trained model to Hugging Face Hub
from google.colab import userdata

REPO_NAME = "TextSummarizerModel"
HF_TOKEN = userdata.get("HF_TOKEN")

# Save the trained model and tokenizer locally
model.save_pretrained(REPO_NAME)
tokenizer.save_pretrained(REPO_NAME)

# Push the model and tokenizer to the Hugging Face Hub
model.push_to_hub(REPO_NAME, token=HF_TOKEN)
tokenizer.push_to_hub(REPO_NAME, token=HF_TOKEN)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/codingP1/TextSummarizerModel/commit/35114cf69429f88ba72be46d4f6bff9ada7da4ef', commit_message='Upload tokenizer', commit_description='', oid='35114cf69429f88ba72be46d4f6bff9ada7da4ef', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Import the necessary modules for running the pipeline
from transformers import pipeline

# Check if a GPU is available and set the appropriate device
import torch
device = 0 if torch.cuda.is_available() else -1

# Load the fine-tuned FLAN-T5 model for text summarization using the Hugging Face pipeline
model = pipeline("summarization", model="codingP1/TextSummarizerModel", device=device)

# Confirm the environment setup and whether the model is running on GPU or CPU
print("Environment set up. Model loaded on:", "GPU" if device == 0 else "CPU")

# Define the long text for summarization
sample_paragraph = """
This is the 24-year-old mother who was allegedly shot dead by her Marine veteran husband
in an Alabama doctor's office after he wrote a chilling Facebook post saying he was going to 'die today'.
Leaj Jarvis Price was reportedly gunned down by Eric Heath Price, the 25-year-old father of her six-year-old son,
on Monday after she ran into the surgery in Jemison screaming 'call the police'. Price then returned to their
nearby home and staged a standoff with around 40 police officers for several hours, authorities said.
When police fired tear gas into the house, he came outside. He was taken to hospital with a self-inflicted
gunshot wound to his head. His condition remains unknown. He has since been charged with the murder of Mrs Price,
who was a nursing student. Scroll down for video. Shot dead: Leaj Jarvis Price (right), a nursing student,
was allegedly shot dead by her Marine veteran husband, Eric Heath Price (pictured, left, kissing Mrs Price)
in an Alabama doctor's office on Monday. Tragic: Mrs Price (pictured with the couple's six-year-old son)
was reportedly gunned down by her spouse, 25, after she ran into the office in Jemison screaming 'call the police'.
...
"""  # (Truncated for brevity)

# Example text input for generating a summary
prompt = "Summarize the following text: " + sample_paragraph

# Generate the summary using the fine-tuned model
response = model(prompt)

# Output the generated summary
print("Generated Summary:", response[0]['summary_text'])


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Environment set up. Model loaded on: CPU
Generated Summary: Leaj Jarvis Price was allegedly shot dead by her Marine veteran husband, Eric Heath Price, on Monday . Price then returned to their nearby home and staged a standoff with 40 police officers for several hours . He has since been charged with the murder of Mrs Price, who was a nursing student .
